In [1]:
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

#wrangling
import pandas as pd
import numpy as np

#explore
import scipy.stats as stats

#visuals
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


#default pandas decimal display formatting
pd.options.display.float_format='{:20,.2f}'.format

import env
import acquire
# import prepare
# import summarize

In [8]:
df = acquire.get_zillow_data()

In [9]:
df.transactiondate = pd.to_datetime(df.transactiondate, format='%Y-%m-%d')
df = df.sort_values("transactiondate").drop_duplicates('parcelid',keep='last') 

In [10]:
df

,id,parcelid,logerror,transactiondate
0,0,14297519,0.03,2017-01-01
1,1,17052889,0.06,2017-01-01
2,2,14186244,0.01,2017-01-01
3,3,12177905,-0.10,2017-01-01
4,4,10887214,0.01,2017-01-01
5,5,17143294,-0.02,2017-01-01
6,6,12095076,-0.00,2017-01-01
7,7,12069064,0.10,2017-01-01
70,70,11018202,0.02,2017-01-02
71,71,10808296,0.01,2017-01-02
